In [8]:
import org.apache.spark

import org.apache.spark


In [ ]:
val sc = spark.SparkContext.getOrCreate()

In [9]:
val path_to_datasets = "../../../datasets/processed"

val path_to_tracks = path_to_datasets + "/tracks.csv"
val path_to_playlists = path_to_datasets + "/playlists.csv"
val path_to_track_in_playlists = path_to_datasets + "/tracks_in_playlist.csv"
val path_to_artists = path_to_datasets + "/artists.csv"

path_to_datasets: String = ../../../datasets/processed
path_to_tracks: String = ../../../datasets/processed/tracks.csv
path_to_playlists: String = ../../../datasets/processed/playlists.csv
path_to_track_in_playlists: String = ../../../datasets/processed/tracks_in_playlist.csv
path_to_artists: String = ../../../datasets/processed/artists.csv


In [10]:
object CsvParser {

  val noGenresListed = "(no genres listed)"
  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val quotes = "\""


  // (PID, playlist_name, num_followers)
  def parsePlayListLine(line: String): Option[(String, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt)
    } catch {
      case _: Exception => None
    }
  }

  // (track_uri, track_name, duration_ms, artist_uri, album_uri, album_name)
  def parseTrackLine(line: String): Option[(String, String, Int, String, String, String)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt, input(3).trim, input(4).trim, input(5).trim)
    } catch {
      case _: Exception => None
    }
  }

  // (artist_uri, artist_name)
  def parseArtistLine(line: String): Option[(String, String)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim)
    } catch {
      case _: Exception => None
    }
  }

  // (PID, track_uri, pos)
  def parseTrackInPlaylistLine(line: String): Option[(String, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt)
    } catch {
      case _: Exception => None
    }
  }
}

defined object CsvParser


In [11]:
val rddTracks = sc.textFile(path_to_tracks).
  flatMap(CsvParser.parseTrackLine)

val rddPlaylists = sc.textFile(path_to_playlists).
  flatMap(CsvParser.parsePlayListLine)

val rddTrackInPlaylists = sc.textFile(path_to_track_in_playlists).
  flatMap(CsvParser.parseTrackInPlaylistLine)

val rddArtists = sc.textFile(path_to_artists).
  flatMap(CsvParser.parseArtistLine)

rddTracks: org.apache.spark.rdd.RDD[(String, String, Int, String, String, String)] = MapPartitionsRDD[9] at flatMap at <console>:33
rddPlaylists: org.apache.spark.rdd.RDD[(String, String, Int)] = MapPartitionsRDD[12] at flatMap at <console>:36
rddTrackInPlaylists: org.apache.spark.rdd.RDD[(String, String, Int)] = MapPartitionsRDD[15] at flatMap at <console>:39
rddArtists: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[18] at flatMap at <console>:42


In [28]:
// ID of the song to analyze
val idSong = "spotify:track:5xlWA3V2l7ZiqYF8Ag5EM8"

idSong: String = spotify:track:0UaMYEvWZi0ZqiDOoHU3YI


In [29]:
// RDD of (pid, trackUri)
val trackInPlaylistReduce = rddTrackInPlaylists.map(x => (x._1, x._2))

trackInPlaylistReduce: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[33] at map at <console>:29


In [30]:
// filter to obtain the id of playlists that contains the track
val playlistForTrack = trackInPlaylistReduce
          .filter { case (_, trackUri) => trackUri == idSong }
          .map(_._1)
          .distinct()

playlistForTrack: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[35] at map at <console>:31


In [31]:
// Broadcast the list of playlists
val playlistsBroadcast = sc.broadcast(playlistForTrack.collect().toSet)

playlistsBroadcast: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Set[String]] = Broadcast(15)


In [32]:
// filter to obtain all the songs in playlist that contains the track
val trackInSamePlaylists = trackInPlaylistReduce
          .filter { case (pid, _) => playlistsBroadcast.value.contains(pid) }
          .filter { case (_, trackUri) => trackUri != idSong }

// RDD of (pid, trackUri)

trackInSamePlaylists: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[37] at filter at <console>:32


In [33]:
// create the pairs of ((mySong, otherSong), 1) for each playlist and reduce by key to count the occurrences
val occurrencesCount = trackInSamePlaylists
  .map { case (_, track) => ((idSong, track), 1) }
  .reduceByKey(_ + _)


rddTrackPairs: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[38] at map at <console>:30


In [35]:
// take the pair with the highest count
val mostOccurrencesPair = occurrencesCount
          .max()(Ordering.by(_._2))

mostOccurrencesPair: ((String, String), Int) = ((spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0XUfyU2QviPAs6bxSpXYG4),253)


In [36]:
// mostOccurrencesPair to rdd to join 
val mostOccurrencesPairRDD = sc.parallelize(Seq(mostOccurrencesPair))

// Unisci i dettagli della traccia specifica e delle tracce correlate
val trackDetails = rddTracks.map(x => (x._1, x._2)) // (trackUri, trackName)

val partitionedTrackDetails = trackDetails.partitionBy(new org.apache.spark.HashPartitioner(trackDetails.getNumPartitions))

val enrichedResults = mostOccurrencesPairRDD
        .map {case ((track1, track2), count) => (track1, (track2, count)) }
        .join(partitionedTrackDetails) // Unisci il nome della traccia principale
        .map { case (track1, ((track2, count), track1Name)) => (track2, (track1, track1Name, count)) }
        .join(partitionedTrackDetails) // Unisci il nome delle co-tracce
        .map { case (track2, ((track1, track1Name, count), track2Name)) => (track1, track1Name, track2, track2Name, count) }

// Salva il risultato
enrichedResults.coalesce(1).saveAsTextFile("output/result")

trackDetails: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[40] at map at <console>:31
mostOccurrencesPairRDD: org.apache.spark.rdd.RDD[((String, String), Int)] = ParallelCollectionRDD[41] at parallelize at <console>:34
enrichedResults: org.apache.spark.rdd.RDD[(String, String, String, String, Int)] = MapPartitionsRDD[50] at map at <console>:41
